# Lab2 - PuLP Library

<b>Information on group members:</b><br>
1) 160306, Piotr Franc <br>
2) 160288, Paweł Charkiewicz

In [2]:
from pulp import *  
import numpy as np
import pandas as pd

## Introduction - brief tutorial on PuLP

In [3]:
# Create an LpProblem instance; LpMaximize = objective function is to be maximized
model = LpProblem(name="some-problem", sense=LpMaximize)

In [4]:
# Initialize the decision variables. We can set the name and lower bound as well.
# To create an array of variables, you can use comprehensions of LpProblem.dicts.

x1 = LpVariable(name="x1", lowBound=0)
x2 = LpVariable(name="x2", lowBound=0)

In [5]:
# An example expression
expression = 3 * x1 + 2 * x2
type(expression)

pulp.pulp.LpAffineExpression

In [6]:
# An example constraint
constraint = 2 * x1 + 3 * x2 >= 5
type(constraint)

pulp.pulp.LpConstraint

Ok, let's now use PuLP to solve the below problem:
$max$ $4x_1 + 2x_2$ <br>
s.t.<br>
     $1x_1 + 1x_2 \geq 1$ <br>
     $2x_1 + 1x_2 \leq 6$ <br>
     $-1x_1 + x_2 = 1$ <br>
     $x_1 \geq 0$ <br>
     $x_2 \geq 0$ 

In [7]:
# Problem
model = LpProblem(name="test-problem", sense=LpMaximize)

x1 = LpVariable(name="x1", lowBound=0)
x2 = LpVariable(name="x2", lowBound=0)

model += (1 * x1 + 1*x2 >= 1, "#1 constraint")
model += (2 * x1 + 1*x2 <= 6, "#2 constraint")
model += (-1 * x1 + 1*x2 == 1, "#3 constraint")

# Objective function
obj_func = 4*x1 + 2 * x2
model += obj_func

model

test-problem:
MAXIMIZE
4*x1 + 2*x2 + 0
SUBJECT TO
#1_constraint: x1 + x2 >= 1

#2_constraint: 2 x1 + x2 <= 6

#3_constraint: - x1 + x2 = 1

VARIABLES
x1 Continuous
x2 Continuous

In [8]:
# Solve the problem
status = model.solve()

In [9]:
# Print status
print(f"status: {model.status}, {LpStatus[model.status]}")

status: 1, Optimal


In [10]:
# Print objective value
print(f"objective: {model.objective.value()}")

objective: 12.000000199999999


In [11]:
# Print constraint values
for name, constraint in model.constraints.items():  print(f"{name}: {constraint.value()}")

#1_constraint: 3.3333334
#2_constraint: 9.999999983634211e-08
#3_constraint: 0.0


In [12]:
model.variables()

[x1, x2]

In [13]:
print(x1.value())
print(x2.value())

1.6666667
2.6666667


### The same code but using dictionaries and other nice tricks


In [14]:
model = LpProblem(name="some-problem", sense=LpMaximize)

In [15]:
var_names = ["First", "Second"]
x = LpVariable.dicts("x", var_names, 0)
x

{'First': x_First, 'Second': x_Second}

In [16]:
const_names = ["GE", "LE", "EQ"]
sense = [1, -1, 0] # GE, LE, EQ
coefs = [[1,1],[2,1],[-1,1]] # Matrix coefs
rhs = [1, 6, 1] 

for c, s, r, cn in zip(coefs, sense, rhs, const_names):
    expr = lpSum([x[var_names[i]] * c[i] for i in range(2)])
    model += LpConstraint(e=expr, sense = s, name = cn, rhs = r)
    
obj_coefs = [4,2]
model += lpSum([x[var_names[i]] * obj_coefs[i] for i in range(2)])
    
model

some-problem:
MAXIMIZE
4*x_First + 2*x_Second + 0.0
SUBJECT TO
GE: x_First + x_Second >= 1

LE: 2 x_First + x_Second <= 6

EQ: - x_First + x_Second = 1

VARIABLES
x_First Continuous
x_Second Continuous

In [17]:
status = model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for n in var_names: print(x[n].value())

status: 1, Optimal
objective: 12.000000199999999
1.6666667
2.6666667


# Homework - use the PuLP library to solve the following OR problem 

Johnson & Sons company manufactures 6 types of toys. Each toy is produced by utilizing at least one Machine 1-4, requiring a different production time (see Table below). For instance, product A requires 4 minutes on Machine 1, 4 minutes on Machine 2, 0 Minutes on Machine 3, and 0 minutes on Machine 4 (all machines must be utilized to produce a toy unless the production time equals 0). Each machine is available for a different number of hours per week. The company aims to identify the number (product-mix) of toys that must be manufactured to maximize the income (can be continuous). Notably, each toy can be sold for a different price. Due to market expectations, the company wants to manufacture twice as many F toys as A. Furthermore, the number of toys B should equal C. Solve this problem using the PuLP library. Prepare a report (in the jupyter notebook) containing information on the following:
<ol>
<li>The number of toys to manufacture;</li>
<li>The expected income;</li>
<li>The production time required on each machine;</li>
</ol>
Consider the total and partial values, i.e., considered for each toy A-F separately (e.g., income resulting from selling toy A). Also, answer the following questions concerning the found solution:
<ol>
<li>Which toy(s) production should be focused on?  </li>
<li>Is there a toy that can be excluded from consideration for production? </li>
<li>Is there a machine that is not fully utilized?</li>
</ol>

| Toy | Machine 1 | Machine 2 | Machine 3 | Machine 4 | Selling price |
| --- | --- | --- | --- | --- | --- |
| A | 4 (minutes!) | 4 | 0 | 0 | 2.50 USD |
| B | 0 | 3 | 3 | 0 | 1.00 USD |
| C | 5 | 0 | 2 | 5 | 4.00 USD |
| D | 2 | 4 | 0 | 4 | 3.00 USD |
| E | 0 | 4 | 5 | 2 | 3.50 USD |
| F | 2 | 2 | 1 | 1 | 4.00 USD |
| Production time limit (hours!) | 120 | 60 |  80 |  120 |  |

In [43]:
model = LpProblem(name="toy-problem", sense=LpMaximize)

a = LpVariable(name="a", lowBound=0, cat="Integer")
b = LpVariable(name="b", lowBound=0, cat="Integer")
c = LpVariable(name="c", lowBound=0, cat="Integer")
d = LpVariable(name="d", lowBound=0, cat="Integer")
e = LpVariable(name="e", lowBound=0, cat="Integer")
f = LpVariable(name="f", lowBound=0, cat="Integer")


model += (4*a + 5*c + 2*d + 2*f <= 120*60, "m1 constraint")
model += (4*a + 3*b + 4*d + 4*e + 2*f <= 60*60, "m2 constraint")
model += (3*b + 2*c + 5*e + f  <= 80*60, "m3 constraint")
model += (5*c + 4*d + 2*e + f <= 120*60, "m4 contraint")

model += (f==2*a, "f and a constraint")
model += (b==c, "b and c constraint")

# Objective function
obj_func = a*2.5 + b*1 + 4*c + 3*d + 3.5*e + 4*f
model += obj_func 

model

status = model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for n in model.variables(): print(n,n.value())

print("Total income:", 2.5*model.variables()[0].value() + 1*model.variables()[1].value() + 4*model.variables()[2].value() + 3*model.variables()[3].value() + 3.5*model.variables()[4].value() + 4*model.variables()[5].value())

print("m1", 4*model.variables()[0].value() + 5*model.variables()[2].value() + 2*model.variables()[3].value() + 2*model.variables()[5].value(), 120*60)
print("m2", 4*model.variables()[0].value() + 3*model.variables()[1].value() + 4*model.variables()[3].value() + 4*model.variables()[4].value() + 2*model.variables()[5].value(), 60*60)
print("m3", 3*model.variables()[1].value() + 2*model.variables()[2].value() + 5*model.variables()[4].value() + model.variables()[5].value(), 80*60)
print("m4", 5*model.variables()[2].value() + 4*model.variables()[3].value() + 2*model.variables()[4].value() + model.variables()[5].value(), 120*60)

status: 1, Optimal
objective: 5698.0
a 106.0
b 917.0
c 917.0
d 0.0
e 0.0
f 212.0
Total income: 5698.0
m1 5433.0 7200
m2 3599.0 3600
m3 4797.0 4800
m4 4797.0 7200


# Report

### Number of toys to manufacture:

| Toy | Number |
|-----------|--------|
| a | 106.0 |
| b | 917.0 |
| c | 917.0 |
| d | 0.0 |
| e | 0.0 |
| f | 212.0 |
| total | 2152|


### Expected income

| Toy | Income |
|-----------|--------|
| a | 265.0 |
| b | 917.0 |
| c | 3668.0 |
| d | 0.0 |
| e | 0.0 |
| f | 848.0 |
| total | 5698.0|

### Production Time on each machine

| Machine | Time used | Total time available |
|----------|--------|-----------|
| m1 | 5433.0 | 7200 |
| m2 | 3599.0 | 3600 |
| m3 | 4797.0 | 4800 |
| m4 | 4797.0 | 7200 |

### Analysis

The production of toys b and c should be focused on. The toys that can be excluded from production are toys d and e as they both heavily rely on machine 2 which has the most limited time available. Machines 1 and 4 arent fully utilized.